## Structure (Dataframe) created:
StockData [ Stock Symbol, Type, Last Dividend, Fixed Dividend, Par Value,Ticker Price]
Trade  [Timestamp, Stock Symbol, Quantity, Buy/Sell, Price]

## Functions:
addTrade(TimeStamp,Symbol,Quantity,Buy_Sell,Price) 
CalculateDividedYield(Symbol)
CalculatePEratio(Symbol)
CalculateStockPrice(Symbol)
CalculateGeometricMean(Symbol)
CalculateGBCE()

In [63]:
# Importing 
import pandas as pd,datetime,calendar,time,random
from scipy.stats.mstats import gmean
# VISUAL
from IPython.display import display

## Create Dataframes:

In [ ]:
StockData = pd.read_csv("StockExample.csv").fillna(0);
StockData.set_index(['Stock Symbol']);
StockData.index = StockData['Stock Symbol'];
StockData['Ticker Price'] = -1

Trade = pd.read_csv("TradeData.csv").fillna(0);
Trade.set_index(['Stock Symbol','Timestamp']);

## Functions

In [57]:
def addTrade(TimeStamp,Symbol,Quantity,Buy_Sell,Price):
    Trade.loc[len(Trade)]=[TimeStamp,Symbol,Quantity,Buy_Sell,Price]
    StockData.loc[(Symbol,'Ticker Price')] = Price

    
def CalculateDividedYield(Symbol):   
    TickerPrice = StockData.loc[(Symbol,'Ticker Price')]
    LastDividedYield = StockData.loc[(Symbol,'Last Dividend')]
    FixedDividend = StockData.loc[(Symbol,'Fixed Dividend')]
    FixedDividend = float(StockData.loc[('GIN','Fixed Dividend')].replace("%",""))  
    
    if(TickerPrice == -1):
        print("! CalculateDividedYield:Warining las TickerPrice of ",Symbol,"not defined")
        return -1
    if(FixedDividend != 0):
        ParValue = StockData.loc[('GIN','Par Value')]
        DividentYieldPreferred = (( float(ParValue) / 100 )  * float(FixedDividend)) / float(TickerPrice)
    if(LastDividedYield == 0):
        print("! CalculateDividedYield:Warining Last Dividend of ",Symbol,"is 0")
        return -1
    
    return TickerPrice / LastDividedYield

def CalculatePEratio(Symbol):   
    TickerPrice = StockData.loc[(Symbol,'Ticker Price')]
    DividedYield = CalculateDividedYield(Symbol)
    
    if(TickerPrice == -1):
        print("! CalculatePEratio:Warining las TickerPrice of ",Symbol,"not defined")
        return -1
    
    if(DividedYield == -1):
        print("! CalculatePEratio:Warining Divided Yield of ",Symbol," not found")
        return -1
    
    if(DividedYield == 0):
        print("! CalculatePEratio:Warining Divided Yield of ",Symbol," is 0")
        return -1
    
    return TickerPrice / DividedYield


def CalculateStockPrice(Symbol):
    Time15MinAgo = calendar.timegm(time.gmtime()) - datetime.timedelta(minutes=15).total_seconds()
    TradeLast15ms = Trade[Trade['Stock Symbol'] == Symbol][Trade['Timestamp'] > Time15MinAgo]
    numTrade = len(TradeLast15ms)

    if(numTrade > 0 ):
        sumTradeXqty = 0
        sumQty = 0
    
        for idx in range(0, numTrade):
            sumTradeXqty += TradeLast15ms.iloc[idx -1]['Price'] * TradeLast15ms.iloc[idx]['Quantity']
            sumQty += TradeLast15ms.iloc[idx-1]['Quantity']
        return sumTradeXqty / sumQty
    else:
        return 0

def CalculateGeometricMean(Symbol):
    return gmean(Trade[Trade['Stock Symbol'] == Symbol]['Price'])

def CalculateGBCE():
    GBCE = {}
    StockSymbols = Trade['Stock Symbol'].unique()
    numSymbols = len(StockSymbols)
    
    for idx in range(0, numSymbols):
        symbol = StockSymbols[idx -1]
        TradeSymbol = Trade[Trade['Stock Symbol'] == symbol]
        numTrade = len(TradeSymbol)
        if(numTrade > 0 ):
            sumTradeXqty = 0
            sumQty = 0
            
            for id2 in range(0, numTrade):
                sumTradeXqty += TradeSymbol.iloc[id2 -1]['Price'] * TradeSymbol.iloc[id2]['Quantity']
                sumQty += TradeSymbol.iloc[id2-1]['Quantity']
            if(sumQty == 0):
                print("! CalculateGBCE:Warining sumQty of ",symbol," is 0")
                GBCE[symbol] = -1
            else:
                GBCE[symbol] = sumTradeXqty / sumQty
    return GBCE

In [ ]:
## Simple running test

In [72]:
# Fast running example
def RecordRandomTrade(numOfTrade):
    for x in range(0, numOfTrade):
        TimeStamp = calendar.timegm(time.gmtime())
        StockSymbols = random.choice(Trade['Stock Symbol'].unique())
        Price = (random.random() * 250)
        Quantity = (random.random() * 100)
        Buy_Sell = random.choice(['Buy','Sell'])
        addTrade(TimeStamp,StockSymbols,Quantity,Buy_Sell,Price)

print("## Add 33 Random Trades")
RecordRandomTrade(33)
ChoosedSymbole = random.choice(Trade['Stock Symbol'].unique())
print("pick a Random symbol for the next steps",ChoosedSymbole)
print("CalculateDividedYield",CalculateDividedYield(ChoosedSymbole))
print("CalculatePEratio",CalculatePEratio(ChoosedSymbole))
print("CalculateStockPrice",CalculateStockPrice(ChoosedSymbole))
print("CalculateGBCE",CalculateGBCE())
display(Trade)
#raise ValueError('Last dividend should be >= 0, not %f')

## Add 33 Random Trades
pick a Random symbol for the next steps TEA
! CalculateDividedYield:Warining Last Dividend of  TEA is 0
CalculateDividedYield -1
! CalculateDividedYield:Warining Last Dividend of  TEA is 0
! CalculatePEratio:Warining Divided Yield of  TEA  not found
CalculatePEratio -1
CalculateStockPrice 124.244021714


/Users/marcoamato/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


CalculateGBCE {'GIN': 109.78786667498441, 'TEA': 118.2812331641921, 'POP': 125.5769873199151, 'JOE': 131.30678702532046, 'ALE': 130.48131857429146}


,Timestamp,Stock Symbol,Quantity,Buy/Sell,Price
0,1.506700e+12,TEA,1.000000,Buy,120.000000
1,1.506700e+12,POP,8.000000,Sell,110.000000
2,1.506700e+12,ALE,23.000000,Buy,60.000000
3,1.506700e+12,GIN,8.000000,Sell,100.000000
4,1.506700e+12,JOE,13.000000,Buy,250.000000
5,1.506700e+09,TEA,1.000000,Buy,60.000000
6,1.506700e+09,POP,8.000000,Sell,100.000000
7,1.506700e+09,ALE,23.000000,Buy,250.000000
8,1.506700e+09,GIN,8.000000,Sell,1.000000
9,1.506700e+09,JOE,13.000000,Buy,2.000000
